In [1]:
from multiprocessing import Pool
import pandas as pd
import numpy as np
import geopandas
import psycopg2
import infostop
import datetime
import time
import os
import random
import logging

In [16]:
logger = logging.getLogger('locations_log')
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('locations.log', mode='w')
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(process)d - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)

In [ ]:
%%time
# path = "Data/devices_json/"
path = "data/locations_json/"
files = os.listdir(path)
len(files)

In [6]:
log = pd.read_csv("locations.log", sep="|", names=['log','file','shape','time'])
#                   names=['datetime', 'name', 'error', 'PID', 'message', 'file', 'errorlog'])
# log.groupby(['errorlog']).count()
# log
log['file'].nunique()

4176

In [5]:
log['file'].nunique()

3742

In [145]:
# cond = log['message'].str.contains('Error', regex=False, na=False)
cond = log['errorlog'].str.contains('No stop', regex=False, na=False)

log[cond][['file']].to_csv("Data/NoStopEventsFound.txt", index=False)

In [141]:
# log[17106:]
log['f']

,datetime,name,error,PID,message,file,errorlog
0,"2021-03-21 15:38:08,448",mongodb_log,INFO,36580,Total files: 1315,NaN,NaN
1,"2021-03-21 15:38:08,565",mongodb_log,ERROR,40564,Error in file:,FILE2A5E844E-EF8C-45C3-BC49-ABD4B6C1CE81.csv,"No stop events found. Check that `r1`, `min_s..."
2,"2021-03-21 15:38:08,565",mongodb_log,ERROR,40695,Error in file:,FILE5E61419F-14AD-454E-B63B-52ECC9F12535.csv,"No stop events found. Check that `r1`, `min_s..."
3,"2021-03-21 15:38:08,569",mongodb_log,ERROR,40615,Error in file:,FILE90ACD969-8B85-4257-83B3-DACD51824787.csv,"No stop events found. Check that `r1`, `min_s..."
4,"2021-03-21 15:38:08,584",mongodb_log,ERROR,40672,Error in file:,FILE04095536-3BC6-4702-8059-0F12C4B08912.csv,"No stop events found. Check that `r1`, `min_s..."
...,...,...,...,...,...,...,...
1318,"2021-03-21 15:38:33,699",mongodb_log,ERROR,340,Error in file:,FILEa5b80ab0-8fbe-4c96-b822-1d4272e28073.csv,"No stop events found. Check that `r1`, `min_s..."
1319,"2021-03-21 15:38:33,772",mongodb_log,ERROR,340,Error in file:,FILE0BCCD917-EEC0-486C-AB9F-64F3FF989B86.csv,"No stop events found. Check that `r1`, `min_s..."
1320,"2021-03-21 15:38:33,854",mongodb_log,ERROR,340,Error in file:,FILE0236E8ED-B6CA-4B22-BAD2-64F45C4E2F8B.csv,"No stop events found. Check that `r1`, `min_s..."
1321,"2021-03-21 15:38:33,955",mongodb_log,ERROR,340,Error in file:,FILEe225a4c9-be64-4e0e-8473-b2fdd312197a.csv,"No stop events found. Check that `r1`, `min_s..."


In [113]:
# cond = log['datetime'] > '2021-03-19 22:30:00'
log.groupby(['message']).count()

,datetime,name,error,PID,file
message,,,,,
'NoneType' object is not subscriptable,24,24,24,24,0
Error in file,49,49,49,49,49
"Error tokenizing data. C error: Expected 4 fields in line 291, saw 5",1,1,1,1,0
File OK,108,108,108,108,108
Latitude (column 0) must have values between -90 and 90,1,1,1,1,0
Longitude (column 1) must have values between -180 and 180,1,1,1,1,0
"No stop events found. Check that `r1`, `min_staying_time` and `min_size` parameters are chosen correctly.",22,22,22,22,0
Parse trajectorie OK,141,141,141,141,141
Total files: 11522,1,1,1,1,0


In [25]:
def process_df(df):
    ts = time.time()
    logger.info('Process Dataframe begin: %s seconds', time.time() - ts)
    df.set_index('device_id', inplace=True)
    df.sort_index(inplace=True)
    
    i = 0
    logger.info("File size {}: ".format(df.shape) )
    users = df.index.unique()
    logger.info("Total unique users: %s", len(users))
    
    for device in users:
        name = "{}/FILE{}.csv".format(path, device)
        if not os.path.isfile(name):
            df.loc[[device]].to_csv(name)
        else:
            df.loc[[device]].to_csv(name, mode='a', header=False)
        i = i + 1
        if i % 10000 == 0:
            logger.info('i: %s users took %s seconds', i, time.time() - ts)
            
    logger.info('Process Dataframe finished: %s seconds', time.time() - ts)

In [5]:
files = [x for x in os.listdir("Data/") if 'part' in x]
files = np.sort(files)
files[1:]

array(['location_part_01', 'location_part_02', 'location_part_03',
       'location_part_04', 'location_part_05', 'location_part_06',
       'location_part_07', 'location_part_08', 'location_part_09',
       'location_part_10', 'location_part_11'], dtype='<U16')

In [ ]:
if __name__ == '__main__':
    ts = time.time()
    pool = Pool(os.cpu_count())                         # Create a multiprocessing Pool

    path = "Data/devices2/"
    
    for f in files[1:]:
        filename = "Data/{}".format(f)
        print(filename)
        chunks = pd.read_csv(filename, chunksize=1000000, names=['datetime', 'device_id', 'x', 'y'])
        pool.map(process_df, chunks)
        print(time.time() - ts)

Data/location_part_01
5113.509140729904
Data/location_part_02
8619.963937282562
Data/location_part_03
12637.965442180634
Data/location_part_04
16242.002480506897
Data/location_part_05
20375.832034111023
Data/location_part_06
24524.784301042557
Data/location_part_07
28650.008158922195
Data/location_part_08
33297.559757232666
Data/location_part_09
37782.6539003849
Data/location_part_10


In [33]:
os.cpu_count()

40

In [37]:
if __name__ == '__main__':
    ts = time.time()
    pool = Pool(os.cpu_count())                         # Create a multiprocessing Pool

    path = "Data/devices/"

    filename = "Data/location_202010.csv"
    print(filename)
    logger.info('Begin Filename: %s', filename)
    chunks = pd.read_csv(filename, chunksize=1000000)
    pool.map(process_df, chunks)
    logger.info('End Filename: %s in %s seconds', filename, time.time() - ts)


Data/location_202010.csv


In [39]:
path = "Data/devices/"
files = os.listdir(path)
len(files)

1911873

In [26]:
files[:10]

['FILE55cdaa35-89e8-4276-aecf-f63978ef1b26.csv',
 'FILEab29f4fd-fca3-4a43-a190-7504f651bb86.csv',
 'FILE43B69818-F06D-4EE1-8DB0-8489FD0C581A.csv',
 'FILEffc6ecf6-a17d-4274-a496-c4ba4c6450cd.csv',
 'FILE3b595c6a-7197-4df4-ac0d-bfcc7f9df33c.csv',
 'FILEb54f42d0-44d0-461f-bb56-4c2505952b5c.csv',
 'FILEa3591ccf-873c-45b7-9373-1f08e31f8461.csv',
 'FILEac93391f-76bc-489e-8ca3-958fb21a7037.csv',
 'FILE3c4404a1-fe9a-4e54-9025-16b93a00972b.csv',
 'FILEa896ffa6-3b38-4114-bfa9-03f21016909b.csv']

In [22]:
filename = files[100000]
filename = os.path.join("Data/devices/", filename)
pd.read_csv(filename)

,device_id,datetime,x,y
0,cb40c95e-d343-459b-bd90-2f84eb22ffce,2020-05-26 00:02:00,-78.528416,-9.126263
1,cb40c95e-d343-459b-bd90-2f84eb22ffce,2020-05-26 00:00:23,-78.528587,-9.126321
2,cb40c95e-d343-459b-bd90-2f84eb22ffce,2020-05-25 16:30:50,-78.528658,-9.126265
3,cb40c95e-d343-459b-bd90-2f84eb22ffce,2020-05-25 22:51:31,-78.528626,-9.126349
4,cb40c95e-d343-459b-bd90-2f84eb22ffce,2020-05-25 20:49:34,-78.528619,-9.126396
...,...,...,...,...
1060,cb40c95e-d343-459b-bd90-2f84eb22ffce,2020-05-25 19:20:30,-78.528621,-9.126316
1061,cb40c95e-d343-459b-bd90-2f84eb22ffce,2020-05-25 16:44:56,-78.528620,-9.126348
1062,cb40c95e-d343-459b-bd90-2f84eb22ffce,2020-05-25 09:59:09,-78.528676,-9.126293
1063,cb40c95e-d343-459b-bd90-2f84eb22ffce,2020-05-25 05:49:19,-78.528616,-9.126325


In [24]:
#pd.read_csv("Data/location_part_01", nrows=1000, names=['datetime', 'device_id', 'x', 'y'])
tmp = pd.read_csv("Data/location_202010.csv", nrows=1000)
tmp
# 1953.0321102142334

,datetime,device_id,x,y
0,2020-10-01 12:09:21,00013F6D-7C0E-4FDE-AFB9-40EC44E907CE,-77.083262,-12.003155
1,2020-10-01 12:11:23,00013F6D-7C0E-4FDE-AFB9-40EC44E907CE,-77.078892,-12.006499
2,2020-10-01 12:12:00,00013F6D-7C0E-4FDE-AFB9-40EC44E907CE,-77.073527,-12.006336
3,2020-10-01 18:24:47,00013F6D-7C0E-4FDE-AFB9-40EC44E907CE,-77.068987,-11.993541
4,2020-10-01 12:11:28,00013F6D-7C0E-4FDE-AFB9-40EC44E907CE,-77.078318,-12.006475
...,...,...,...,...
995,2020-10-01 18:04:59,018083EF-A151-46EA-A8BE-2A4F8D88019B,-77.039534,-12.094729
996,2020-10-01 13:50:39,018083EF-A151-46EA-A8BE-2A4F8D88019B,-77.039485,-12.094829
997,2020-10-01 16:20:44,018083EF-A151-46EA-A8BE-2A4F8D88019B,-77.039616,-12.095010
998,2020-10-01 00:31:18,018083EF-A151-46EA-A8BE-2A4F8D88019B,-77.005223,-12.132984


In [17]:
files = [x for x in os.listdir("Data/") if 'part' in x]
files.sort

<function list.sort>

In [3]:
devlist = pd.read_csv("Data/devices.txt", names=['device_id'])
devlist['filename'] = devlist.apply(lambda x: 'FILE'+x.device_id+'.csv', axis=1)
devlist

,device_id,filename
0,000044e3-5b64-415c-97c7-5ff91f35ded0,FILE000044e3-5b64-415c-97c7-5ff91f35ded0.csv
1,00004999-1887-4158-95da-de538ab17f13,FILE00004999-1887-4158-95da-de538ab17f13.csv
2,00005c55-aae7-48dc-82b2-08a56e02f792,FILE00005c55-aae7-48dc-82b2-08a56e02f792.csv
3,00007e9a-f0e3-4a5f-9c91-842ed16db0bb,FILE00007e9a-f0e3-4a5f-9c91-842ed16db0bb.csv
4,0000bb86-fe13-425b-8dbd-adcbbc040d6f,FILE0000bb86-fe13-425b-8dbd-adcbbc040d6f.csv
...,...,...
527517,ffff1311-8fdb-4986-9669-d565664ef065,FILEffff1311-8fdb-4986-9669-d565664ef065.csv
527518,ffff2765-0e4a-4781-9188-b8f0edff63ce,FILEffff2765-0e4a-4781-9188-b8f0edff63ce.csv
527519,ffff4858-b1e3-40ac-8616-abe91b550531,FILEffff4858-b1e3-40ac-8616-abe91b550531.csv
527520,ffffbbcb-a7af-4e58-bc6e-6036a368cc20,FILEffffbbcb-a7af-4e58-bc6e-6036a368cc20.csv


In [4]:
path = "Data/devices_list/"
files = os.listdir(path)
len(files)

505914

In [27]:
gap = list(set(devlist['filename']) - set(files))
gap = pd.DataFrame(gap)
gap.head()

,0
0,FILE35960847-2979-4d40-b60a-a3a604cbaa2a.csv
1,FILE574c1da4-6073-474c-84f2-17af70e63f36.csv
2,FILE0c6722a4-f485-4317-a69e-504c32265790.csv
3,FILE9e21f1af-5993-40e6-a55c-4cf31e831ecb.csv
4,FILEe1d844f9-b919-4f44-8f55-9f132d257e76.csv


In [36]:
gap['filename'] = gap.apply(lambda x: x['filename'].upper().replace('.CSV', '.csv'), axis=1)

In [37]:
gap.head()

,filename
0,FILE35960847-2979-4D40-B60A-A3A604CBAA2A.csv
1,FILE574C1DA4-6073-474C-84F2-17AF70E63F36.csv
2,FILE0C6722A4-F485-4317-A69E-504C32265790.csv
3,FILE9E21F1AF-5993-40E6-A55C-4CF31E831ECB.csv
4,FILEE1D844F9-B919-4F44-8F55-9F132D257E76.csv


In [8]:
cad = 'FILEa185f072-e0c5-4c88-afb6-da55b1d63f86.csv'
cad.upper()

'FILEA185F072-E0C5-4C88-AFB6-DA55B1D63F86.CSV'

In [9]:
len(gap)
pd.read_csv("Data/devices/FILEA185F072-E0C5-4C88-AFB6-DA55B1D63F86.csv", sep=";")

,"device_id,datetime,x,y"
0,"A185F072-E0C5-4C88-AFB6-DA55B1D63F86,2020-06-0..."
1,"A185F072-E0C5-4C88-AFB6-DA55B1D63F86,2020-06-2..."
2,"A185F072-E0C5-4C88-AFB6-DA55B1D63F86,2020-06-2..."
3,"A185F072-E0C5-4C88-AFB6-DA55B1D63F86,2020-06-2..."
4,"A185F072-E0C5-4C88-AFB6-DA55B1D63F86,2020-06-2..."
...,...
96,"A185F072-E0C5-4C88-AFB6-DA55B1D63F86,2020-09-2..."
97,"A185F072-E0C5-4C88-AFB6-DA55B1D63F86,2020-09-2..."
98,"A185F072-E0C5-4C88-AFB6-DA55B1D63F86,2020-09-2..."
99,"A185F072-E0C5-4C88-AFB6-DA55B1D63F86,2020-09-2..."


In [38]:
def copyFile (file):
# file = 'FILE55cdaa35-89e8-4276-aecf-f63978ef1b26.csv'
#     file = file.upper()
    origin = os.path.join("Data/devices", file)
    destiny = os.path.join("Data/devices_list", file)

    cmd = "cp {} {}".format(origin, destiny)
    os.system(cmd)

In [10]:
# x = os.listdir("Data/devices_list")
gap

['FILE35960847-2979-4d40-b60a-a3a604cbaa2a.csv',
 'FILE574c1da4-6073-474c-84f2-17af70e63f36.csv',
 'FILE0c6722a4-f485-4317-a69e-504c32265790.csv',
 'FILE9e21f1af-5993-40e6-a55c-4cf31e831ecb.csv',
 'FILEe1d844f9-b919-4f44-8f55-9f132d257e76.csv',
 'FILE0260729e-1849-4737-9411-8e1a84100116.csv',
 'FILE0be7576a-1c4e-40f9-b319-28560c061cf0.csv',
 'FILE933a76ad-d704-4dd8-a860-25e64366913c.csv',
 'FILE87172af2-061b-4561-9812-44b3c49882b7.csv',
 'FILEd117cd62-ec12-4467-9c5b-e1e07c9d5a09.csv',
 'FILE7c9e962c-814e-4535-aa98-0ab5e0085ffd.csv',
 'FILE96939b63-bf52-4518-a3c6-399415413209.csv',
 'FILE637ee3db-fe16-4346-8bf2-ef12da070f50.csv',
 'FILEe5c2c8ef-0667-42f0-8f0f-e3c8705d4211.csv',
 'FILE20c273b3-3d21-4340-b468-0d53cf4c15c9.csv',
 'FILE453d7180-3b69-45fe-991a-27536c757b79.csv',
 'FILEa185f072-e0c5-4c88-afb6-da55b1d63f86.csv',
 'FILEe3469212-1b25-4b5e-bbda-0d7ba7f68915.csv',
 'FILEd7fedf6c-56c5-4308-9b55-50ca793d2ce0.csv',
 'FILE8a41c1d5-acfe-4b82-8e75-f9618427872c.csv',
 'FILE32da92e9-1d84-

In [40]:
if __name__ == '__main__':
    ts = time.time()
    pool = Pool(os.cpu_count())                         # Create a multiprocessing Pool

#     chunks = devlist['filename']
    chunks = gap['filename']
    pool.map(copyFile, chunks)
    logger.info('Copy files in %s seconds', time.time() - ts)
    print(time.time() - ts)


919.9082708358765


In [39]:
gap['filename']

0        FILE35960847-2979-4D40-B60A-A3A604CBAA2A.csv
1        FILE574C1DA4-6073-474C-84F2-17AF70E63F36.csv
2        FILE0C6722A4-F485-4317-A69E-504C32265790.csv
3        FILE9E21F1AF-5993-40E6-A55C-4CF31E831ECB.csv
4        FILEE1D844F9-B919-4F44-8F55-9F132D257E76.csv
                             ...                     
21603    FILED4066284-8FFD-4484-8E40-8FC8661772E3.csv
21604    FILEAF713BD6-F55E-4C0E-B161-E53C68FE3DF8.csv
21605    FILE6D640D3A-C4B9-43CF-B8A2-75CD552D7D80.csv
21606    FILE411C29CC-C01D-4494-BE1E-CEAEA9E13B63.csv
21607    FILED80973BD-2D1A-414F-8C44-133B0455F01C.csv
Name: filename, Length: 21608, dtype: object

In [26]:
pd.DataFrame(gap, names=['filename'])

TypeError: __init__() got an unexpected keyword argument 'names'